In [33]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [2]:
(train_pool_X, train_pool_y), (test_X, test_y) = tf.keras.datasets.mnist.load_data()
train_X, Pool_X, train_y,  Pool_y = train_test_split(train_pool_X, train_pool_y, train_size=5, random_state=42)
print(train_X.shape, Pool_X.shape, train_y.shape, Pool_y.shape, test_X.shape, test_y.shape)
train_pool_X = train_pool_X.reshape(train_pool_X.shape[0], -1)
train_X = train_X.reshape(train_X.shape[0], -1)
test_X = test_X.reshape(test_X.shape[0], -1)
Pool_X = Pool_X.reshape(Pool_X.shape[0], -1)
print(train_pool_X.shape, train_X.shape, test_X.shape, Pool_X.shape)

(50, 28, 28) (59950, 28, 28) (50,) (59950,) (10000, 28, 28) (10000,)
(60000, 784) (50, 784) (10000, 784) (59950, 784)


In [23]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN,self).__init__()
        self.fc1 = nn.Linear(28*28,128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128,10)
        self.dropout = nn.Dropout(0.5)

    def forward(self,x):
        x = x.view(-1,28*28)
        x= F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Assuming your dataset is in NumPy arrays
# train_X, train_y, test_X, test_y, Pool_X, Pool_y = ...

# Reshape the data
train_X = train_X.reshape(train_X.shape[0], -1)
test_X = test_X.reshape(test_X.shape[0], -1)
Pool_X = Pool_X.reshape(Pool_X.shape[0], -1)

# Convert NumPy arrays to PyTorch tensors
train_X_tensor = torch.tensor(train_X, dtype=torch.float32)
train_y_tensor = torch.tensor(train_y, dtype=torch.long)
test_X_tensor = torch.tensor(test_X, dtype=torch.float32)
test_y_tensor = torch.tensor(test_y, dtype=torch.long)
Pool_X_tensor = torch.tensor(Pool_X, dtype=torch.float32)

# Define batch size for DataLoader
batch_size = 64

# Create DataLoader for the unlabeled dataset (Pool_X_tensor)
unlabeled_dataset = TensorDataset(Pool_X_tensor)
unlabeled_data_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=True)

# Define your SimpleNN model class
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, enable_dropout=False):
        x = F.relu(self.fc1(x))
        if enable_dropout:
            x = self.dropout(x)
        x = self.fc2(x)
        return x

    def enable_dropout(self):
        """
        Enable dropout layers during model evaluation.
        """
        for module in self.modules():
            if isinstance(module, nn.Dropout):
                module.train()

# Instantiate the SimpleNN model
model = SimpleNN()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_data_loader):
        # Reshape images if necessary
        images = images.view(-1, 28*28)

        # Move images and labels to the appropriate device (e.g., GPU)
        images = images.to(torch.device('cuda'))
        labels = labels.to(torch.device('cuda'))

        # Forward pass
        outputs = model(images)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training progress
        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_data_loader)}], Loss: {loss.item():.4f}")

# Active Learning with Epistemic Uncertainty using Monte Carlo Dropout
import numpy as np
import torch
import torch.nn as nn

def get_monte_carlo_predictions(data_loader, forward_passes, model, n_classes, n_samples):

    dropout_predictions = np.empty((0, n_samples, n_classes))
    softmax = nn.Softmax(dim=1)
    for i in range(forward_passes):
        predictions = np.empty((0, n_classes))
        model.eval()
        enable_dropout(model)
        for i, image in enumerate(data_loader):
            image = image.to(torch.device('cuda'))
            with torch.no_grad():
                output = model(image)
                output = softmax(output)  
            predictions = np.vstack((predictions, output.cpu().numpy()))

        dropout_predictions = np.vstack((dropout_predictions, predictions[np.newaxis, :, :]))

    mean = np.mean(dropout_predictions, axis=0)

    return mean

def enable_dropout(model):
    for module in model.modules():
        if isinstance(module, nn.Dropout):
            module.train()

def entropy(p):
    return -np.sum(p * np.log2(p + 1e-8), axis=1)  # Adding a small value to avoid log(0) errors

def active_learning_with_uncertainty(data_loader, forward_passes, model, n_classes, n_samples, alpha=0.5, num_samples_to_label=100):
    dropout_predictions = get_monte_carlo_predictions(data_loader, forward_passes, model, n_classes, n_samples)

    # Calculate epistemic uncertainty using variance of predictions
    variance = np.var(dropout_predictions, axis=0)

    # Calculate entropy of the predicted probabilities
    mean = np.mean(dropout_predictions, axis=0)
    entropies = entropy(mean)

    # Calculate the uncertainty score (a weighted combination of variance and entropy)
    uncertainty_score = alpha * variance + (1 - alpha) * entropies

    # Sort the samples based on the calculated uncertainty scores
    sorted_indices = np.argsort(uncertainty_score)

    # Select the most uncertain samples for labeling
    selected_indices = sorted_indices[-num_samples_to_label:]

    return selected_indices

# Active Learning Loop
num_iterations = 5
num_samples_to_label = 100
forward_passes = 10  # You can adjust the number of forward passes as per your requirements

for iteration in range(num_iterations):
    print(f"Active Learning Iteration {iteration + 1}/{num_iterations}")
    
    # Step 1: Select the most uncertain samples from the unlabeled dataset
    selected_indices = active_learning_with_uncertainty(
        data_loader=unlabeled_data_loader,
        forward_passes=forward_passes,
        model=model,
        n_classes=10,  # Replace with the actual number of classes in your dataset
        n_samples=num_samples_to_label
    )

    # Step 2: Label the selected samples and add them to the training dataset
    labeled_images = Pool_X_tensor[selected_indices]
    labeled_labels = Pool_y_tensor[selected_indices]

    # Combine the labeled samples with the original training dataset
    train_X_tensor = torch.cat([train_X_tensor, labeled_images])
    train_y_tensor = torch.cat([train_y_tensor, labeled_labels])

    # Create a new DataLoader for the updated training dataset
    train_dataset = TensorDataset(train_X_tensor, train_y_tensor)
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Step 3: Retrain the model on the updated training dataset
    for epoch in range(num_epochs):
        model.train()
        for i, (images, labels) in enumerate(train_data_loader):
            # Reshape images if necessary
            images = images.view(-1, 28*28)

            # Move images and labels to the appropriate device (e.g., GPU)
            images = images.to(torch.device('cuda'))
            labels = labels.to(torch.device('cuda'))

            # Forward pass
            outputs = model(images)

            # Compute the loss
            loss = criterion(outputs, labels)

            # Backpropagation and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Print training progress
            if (i + 1) % 100 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_data_loader)}], Loss: {loss.item():.4f}")

# After the active learning iterations, your model should be trained on the updated dataset.
# You can now evaluate the model on the test dataset to measure its performance.
model.eval()
with torch.no_grad():
    test_X_tensor = test_X_tensor.to(torch.device('cuda'))
    test_y_tensor = test_y_tensor.to(torch.device('cuda'))
    outputs = model(test_X_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == test_y_tensor).sum().item() / len(test_y_tensor)
    print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
def enable_dropout(model):
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

In [ ]:
def get_monte_carlo_predictions(data_loader,
                                forward_passes,
                                model,
                                n_classes,
                                n_samples):

    dropout_predictions = np.empty((0, n_samples, n_classes))
    softmax = nn.Softmax(dim=1)
    for i in range(forward_passes):
        predictions = np.empty((0, n_classes))
        model.eval()
        enable_dropout(model)
        for i, (image, label) in enumerate(data_loader):
            image = image.to(torch.device('cuda'))
            with torch.no_grad():
                output = model(image)
                output = softmax(output)  
            predictions = np.vstack((predictions, output.cpu().numpy()))

        dropout_predictions = np.vstack((dropout_predictions,
                                         predictions[np.newaxis, :, :]))

    mean = np.mean(dropout_predictions, axis=0)


In [ ]:
def entropy(p):
    return -np.sum(p * np.log2(p), axis=1)

In [24]:
def mc_dropout_predictions(model, X, num_samples=10):
    model.train()
    predictions = []
    with torch.no_grad():
        for _ in range(num_samples):
            output = model(X)
            predictions.append(output.softmax(dim=1).numpy())
    return np.array(predictions)

In [26]:
def calculate_accuracy(model, test_X, test_y):
    model.eval() 
    with torch.no_grad():
        test_X_tensor = torch.Tensor(test_X).to(torch.float32)
        outputs = model(test_X_tensor)
        _, predicted_labels = torch.max(outputs, 1)
        correct = (predicted_labels == torch.Tensor(test_y)).sum().item()
        total = len(test_y)
        accuracy = correct / total
        # print('Accuracy: %.2f' % (accuracy*100))
    return accuracy

In [46]:
def active_learning(train_X,train_y,pool_X_train,pool_y_train,pool_X_test,y_test,acquisition_function,num_iterations,number_of_sample=10):
    model=SimpleNN()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    new_indices_list=[]
    train_accuracies_al=[]
    test_accuracies_al=[]
    for i in range(num_iterations):
        model.train()
        optimizer.zero_grad()
        output = model(torch.Tensor(pool_X_train).to(torch.float32))
        probablities = F.softmax(output, dim=1).detach().numpy()
        acquisition_score=acquisition_function(probablities)
        new_indices = np.argsort(acquisition_score)[::-1][:number_of_sample]
        new_indices_list.append(new_indices)
        labeled_x=pool_X_train[new_indices]
        labeled_y=pool_y_train[new_indices]
        train_X=np.concatenate((train_X,labeled_x),axis=0)
        train_y=np.concatenate((train_y,labeled_y),axis=0)
        pool_X_train=np.delete(pool_X_train,new_indices,axis=0)
        pool_y_train=np.delete(pool_y_train,new_indices)
        train_X_tensor=torch.Tensor(train_X).to(torch.float32)
        train_y_tensor=torch.Tensor(train_y).to(torch.long)
        output = model(train_X_tensor)
        loss = criterion(output, train_y_tensor)
        loss.backward()
        optimizer.step()
        
    return train_accuracies_al,test_accuracies_al,new_indices_list    

In [36]:
def active_learning_mc_dropout(train_X, train_y, pool_X_train, pool_y_train, pool_X_test, y_test, num_iterations, num_samples=10):
    model=SimpleNN()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    train_accuracies = []
    test_accuracies = []   
    new_indices = []
    for i in range(num_iterations):
        model.train()
        optimizer.zero_grad() 
        output = mc_dropout_predictions(model,torch.Tensor(pool_X_train), num_samples)
        uncertainty = np.var(output, axis=0).sum(axis=1)
        new_index = np.argsort(uncertainty)[-1:]
        new_indices.append(new_index)
        label_X = pool_X_train[new_index]
        label_y = pool_y_train[new_index]
        train_X = np.concatenate((train_X, label_X), axis=0)
        train_y = np.concatenate((train_y, label_y), axis=0)
        pool_X_train = np.delete(pool_X_train, new_index, axis=0)
        pool_y_train = np.delete(pool_y_train, new_index)
        train_X_tensor = torch.Tensor(train_X).to(torch.float32)
        train_y_tensor = torch.Tensor(train_y).to(torch.long)
        output = model(train_X_tensor)
        loss = criterion(output, train_y_tensor)
        loss.backward()
        optimizer.step()
        train_accuracy = calculate_accuracy(model, train_X_tensor, train_y_tensor)
        train_accuracies.append(train_accuracy)
        model.eval()
        test_accuracy = calculate_accuracy(model, torch.Tensor(pool_X_test).to(torch.float32), y_test)
        test_accuracies.append(test_accuracy)
    return train_accuracies, test_accuracies, new_indices
    


In [50]:
train_accuracies_al, test_accuracies_al,new_indice_entropy = active_learning(train_X, train_y, Pool_X, Pool_y, test_X, test_y, acquisition_function=entropy, num_iterations=50)


/tmp/ipykernel_2059/3723900648.py:2: RuntimeWarning: divide by zero encountered in log2
  return -np.sum(p * np.log2(p), axis=1)
/tmp/ipykernel_2059/3723900648.py:2: RuntimeWarning: invalid value encountered in multiply
  return -np.sum(p * np.log2(p), axis=1)


Accuracy: 36.67
Accuracy: 25.99
Accuracy: 54.29
Accuracy: 33.27
Accuracy: 68.75
Accuracy: 40.87
Accuracy: 78.89
Accuracy: 49.95
Accuracy: 84.00
Accuracy: 57.89
Accuracy: 89.09
Accuracy: 61.33
Accuracy: 90.83
Accuracy: 63.02
Accuracy: 90.00
Accuracy: 63.31
Accuracy: 92.86
Accuracy: 63.31
Accuracy: 94.67
Accuracy: 64.10
Accuracy: 94.38
Accuracy: 64.74
Accuracy: 96.47
Accuracy: 65.39
Accuracy: 97.78
Accuracy: 65.60
Accuracy: 96.84
Accuracy: 65.88
Accuracy: 97.00
Accuracy: 66.40
Accuracy: 96.67
Accuracy: 67.00
Accuracy: 96.82
Accuracy: 67.31
Accuracy: 97.39
Accuracy: 67.59
Accuracy: 97.92
Accuracy: 67.95
Accuracy: 98.00
Accuracy: 68.10
Accuracy: 98.85
Accuracy: 68.33
Accuracy: 98.89
Accuracy: 68.42
Accuracy: 99.64
Accuracy: 68.45
Accuracy: 99.66
Accuracy: 68.71
Accuracy: 100.00
Accuracy: 68.69
Accuracy: 100.00
Accuracy: 68.88
Accuracy: 99.69
Accuracy: 68.77
Accuracy: 99.70
Accuracy: 69.11
Accuracy: 99.71
Accuracy: 69.22
Accuracy: 99.71
Accuracy: 69.41
Accuracy: 99.72
Accuracy: 69.51
Accura

In [51]:
train_accuracies, test_accuracies, new_indices_list_mc_dropout = active_learning_mc_dropout(train_X, train_y, Pool_X, Pool_y, test_X, test_y, num_iterations=50)


Accuracy: 37.25
Accuracy: 18.81
Accuracy: 51.92
Accuracy: 31.70
Accuracy: 71.70
Accuracy: 39.22
Accuracy: 85.19
Accuracy: 44.86
Accuracy: 87.27
Accuracy: 48.73
Accuracy: 94.64
Accuracy: 51.17
Accuracy: 98.25
Accuracy: 51.56
Accuracy: 100.00
Accuracy: 51.84
Accuracy: 100.00
Accuracy: 52.83
Accuracy: 98.33
Accuracy: 54.59
Accuracy: 96.72
Accuracy: 56.86
Accuracy: 93.55
Accuracy: 58.42
Accuracy: 92.06
Accuracy: 59.33
Accuracy: 93.75
Accuracy: 60.12
Accuracy: 95.38
Accuracy: 60.95
Accuracy: 95.45
Accuracy: 61.42
Accuracy: 95.52
Accuracy: 61.91
Accuracy: 95.59
Accuracy: 62.16
Accuracy: 97.10
Accuracy: 62.30
Accuracy: 97.14
Accuracy: 62.38
Accuracy: 97.18
Accuracy: 62.22
Accuracy: 95.83
Accuracy: 61.91
Accuracy: 95.89
Accuracy: 61.69
Accuracy: 95.95
Accuracy: 61.38
Accuracy: 94.67
Accuracy: 61.33
Accuracy: 94.74
Accuracy: 61.39
Accuracy: 94.81
Accuracy: 61.57
Accuracy: 93.59
Accuracy: 61.75
Accuracy: 93.67
Accuracy: 61.61
Accuracy: 96.25
Accuracy: 61.73
Accuracy: 96.30
Accuracy: 61.67
Accura

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Assuming your dataset is in NumPy arrays
# train_X, train_y, test_X, test_y, Pool_X, Pool_y = ...

# Reshape the data
train_X = train_X.reshape(train_X.shape[0], -1)
test_X = test_X.reshape(test_X.shape[0], -1)
Pool_X = Pool_X.reshape(Pool_X.shape[0], -1)

# Convert NumPy arrays to PyTorch tensors
train_X_tensor = torch.tensor(train_X, dtype=torch.float32)
train_y_tensor = torch.tensor(train_y, dtype=torch.long)
test_X_tensor = torch.tensor(test_X, dtype=torch.float32)
test_y_tensor = torch.tensor(test_y, dtype=torch.long)
Pool_X_tensor = torch.tensor(Pool_X, dtype=torch.float32)

# Define batch size for DataLoader
batch_size = 64

# Create DataLoader for the unlabeled dataset (Pool_X_tensor)
unlabeled_dataset = TensorDataset(Pool_X_tensor)
unlabeled_data_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=True)

# Define your SimpleNN model class
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, enable_dropout=False):
        x = F.relu(self.fc1(x))
        if enable_dropout:
            x = self.dropout(x)
        x = self.fc2(x)
        return x

    def enable_dropout(self):
        """
        Enable dropout layers during model evaluation.
        """
        for module in self.modules():
            if isinstance(module, nn.Dropout):
                module.train()

# Instantiate the SimpleNN model
model = SimpleNN()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_data_loader):
        # Reshape images if necessary
        images = images.view(-1, 28*28)

        # Move images and labels to the appropriate device (e.g., GPU)
        images = images.to(torch.device('cuda'))
        labels = labels.to(torch.device('cuda'))

        # Forward pass
        outputs = model(images)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training progress
        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_data_loader)}], Loss: {loss.item():.4f}")

# Active Learning with Epistemic Uncertainty using Monte Carlo Dropout
import numpy as np
import torch
import torch.nn as nn

def get_monte_carlo_predictions(data_loader, forward_passes, model, n_classes, n_samples):

    dropout_predictions = np.empty((0, n_samples, n_classes))
    softmax = nn.Softmax(dim=1)
    for i in range(forward_passes):
        predictions = np.empty((0, n_classes))
        model.eval()
        enable_dropout(model)
        for i, image in enumerate(data_loader):
            image = image.to(torch.device('cuda'))
            with torch.no_grad():
                output = model(image)
                output = softmax(output)  
            predictions = np.vstack((predictions, output.cpu().numpy()))

        dropout_predictions = np.vstack((dropout_predictions, predictions[np.newaxis, :, :]))

    mean = np.mean(dropout_predictions, axis=0)

    return mean

def enable_dropout(model):
    for module in model.modules():
        if isinstance(module, nn.Dropout):
            module.train()

def entropy(p):
    return -np.sum(p * np.log2(p + 1e-8), axis=1)  # Adding a small value to avoid log(0) errors

def active_learning_with_uncertainty(data_loader, forward_passes, model, n_classes, n_samples, alpha=0.5, num_samples_to_label=100):
    dropout_predictions = get_monte_carlo_predictions(data_loader, forward_passes, model, n_classes, n_samples)

    # Calculate epistemic uncertainty using variance of predictions
    variance = np.var(dropout_predictions, axis=0)

    # Calculate entropy of the predicted probabilities
    mean = np.mean(dropout_predictions, axis=0)
    entropies = entropy(mean)

    # Calculate the uncertainty score (a weighted combination of variance and entropy)
    uncertainty_score = alpha * variance + (1 - alpha) * entropies

    # Sort the samples based on the calculated uncertainty scores
    sorted_indices = np.argsort(uncertainty_score)

    # Select the most uncertain samples for labeling
    selected_indices = sorted_indices[-num_samples_to_label:]

    return selected_indices

# Active Learning Loop
num_iterations = 5
num_samples_to_label = 100
forward_passes = 10  # You can adjust the number of forward passes as per your requirements

for iteration in range(num_iterations):
    print(f"Active Learning Iteration {iteration + 1}/{num_iterations}")
    
    # Step 1: Select the most uncertain samples from the unlabeled dataset
    selected_indices = active_learning_with_uncertainty(
        data_loader=unlabeled_data_loader,
        forward_passes=forward_passes,
        model=model,
        n_classes=10,  # Replace with the actual number of classes in your dataset
        n_samples=num_samples_to_label
    )

    # Step 2: Label the selected samples and add them to the training dataset
    labeled_images = Pool_X_tensor[selected_indices]
    labeled_labels = Pool_y_tensor[selected_indices]

    # Combine the labeled samples with the original training dataset
    train_X_tensor = torch.cat([train_X_tensor, labeled_images])
    train_y_tensor = torch.cat([train_y_tensor, labeled_labels])

    # Create a new DataLoader for the updated training dataset
    train_dataset = TensorDataset(train_X_tensor, train_y_tensor)
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Step 3: Retrain the model on the updated training dataset
    for epoch in range(num_epochs):
        model.train()
        for i, (images, labels) in enumerate(train_data_loader):
            # Reshape images if necessary
            images = images.view(-1, 28*28)

            # Move images and labels to the appropriate device (e.g., GPU)
            images = images.to(torch.device('cuda'))
            labels = labels.to(torch.device('cuda'))

            # Forward pass
            outputs = model(images)

            # Compute the loss
            loss = criterion(outputs, labels)

            # Backpropagation and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Print training progress
            if (i + 1) % 100 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_data_loader)}], Loss: {loss.item():.4f}")

# After the active learning iterations, your model should be trained on the updated dataset.
# You can now evaluate the model on the test dataset to measure its performance.
model.eval()
with torch.no_grad():
    test_X_tensor = test_X_tensor.to(torch.device('cuda'))
    test_y_tensor = test_y_tensor.to(torch.device('cuda'))
    outputs = model(test_X_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == test_y_tensor).sum().item() / len(test_y_tensor)
    print(f"Test Accuracy: {accuracy:.4f}")
